In [1]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import numpy as np

import sys
sys.path.append('functions/')

set_matplotlib_formats('svg')
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# File browser:

In [2]:
from filebrowser import FileBrowser
File = FileBrowser()
display(File.widget())

path = File.path

# Functions

In [3]:
def demo(Index, Nz, Reg_L1, Reg_L2, Reg_C, Reg_S, Bounds, dt, Methods, Plot, LCurve, Residuals, Heatplot):
    """Gets data from interface() and display p\rocessed data

    Index     – int value contains an index of transient in dataset
    Nz        – int value which is lenght of calculated vector
    Reg_L1, Reg_L2 – reg. parameters for L1 and L2 regularisation
    Bounds    – list with left and right bound of t-domain
    Methods   – list with methods to process data
    Plot      – boolean which calls plot_data() if true
    Residuals – (not working yet)
    Hetplot   – plots heatplot for all dataset

    """
    import numpy as np
    from read_file import read_file
    from laplace import laplace
    from plot_data import plot_data
    from hp import hp
    from read_file import read_file
    from residuals import residuals

    Bounds = 10.0**np.asarray(Bounds)

    s, C, T = read_file(File.path, dt)# time, transients, temperatures 
    
    data = laplace(s, C[Index], Nz, Reg_L1, Reg_L2, Reg_C, Reg_S, Bounds, Methods)
    #print(data)
    
    if Plot:
        ay = plot_data(s, C[Index], data, T, Index)
        
    if Residuals:
        residuals(s, C[Index], ay, Methods, Reg_L1, Reg_L2, Reg_C, Reg_S, Bounds, Nz)
    
    if Heatplot:
        hp(s, C, T, Methods, Index, Reg_L1, Reg_L2, Reg_C, Reg_S, Bounds, Nz, LCurve)

In [4]:
def interface():

    '''Initiates widgets'''

    from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Label
    import ipywidgets as widgets
    import numpy as np
    from read_file import read_file
    import warnings
    warnings.filterwarnings('ignore')
    

    t, C, T = read_file(File.path)
    
    if len(T.shape) != 0:
        cut = len(T) - 1
    else:
        cut = 1
    Index = widgets.IntSlider(
        value=0,
        min=0, # max exponent of base
        max=cut, # min exponent of base
        step=1, # exponent step
        description='')

    Methods = widgets.SelectMultiple(
        options = ['L1', 'L2', 'L1+L2', 'Contin', 'reSpect'],
        value   = ['Contin'],
        #rows    = 10,
        description = 'Methods:',
        disabled = False)

    Nz = widgets.IntText(
        value=100,
        description=r'$N_f=$',
        disabled=False)

    Reg_L1 = widgets.FloatLogSlider(
        value=0.1,
        base=10,
        min=-5, # max exponent of base
        max=1, # min exponent of base
        step=0.2, # exponent step
        description=r'L1: $\lambda_1$')

    Reg_L2 = widgets.FloatLogSlider(
        value=1E-0,
        base=10,
        min=-4, # max exponent of base
        max=2, # min exponent of base
        step=0.2, # exponent step
        description=r'L2: $\lambda_2$')
    
    Reg_C = widgets.FloatLogSlider(
        value=1E-1,
        base=10,
        min=-8, # max exponent of base
        max=2, # min exponent of base
        step=0.2, # exponent step
        description=r'Contin: $\lambda_{\text{C}}$')
    
    Reg_S = widgets.FloatLogSlider(
        value=1E-2,
        base=10,
        min=-12, # max exponent of base
        max=2, # min exponent of base
        step=0.2, # exponent step
        description=r'reSpect: $\lambda_{\text{S}}$')

    Bounds = widgets.IntRangeSlider(
        value=[-2, 2],
        min=-5,
        max=5,
        step=1,
        description=r'$10^{a}\div 10^{b}$:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d')
    
    dt = widgets.BoundedFloatText(
        value=150,
        min=1,
        max=1000,
        step=1,
        description='Time step, ms',
        disabled=False)

    Plot = widgets.Checkbox(
        value = True,
        description = 'Plot graph?',
        disabled = False)
    
    LCurve = widgets.Checkbox(
        value = False,
        description = 'Use L-Curve optimal?',
        disabled = False)

    Residuals = widgets.ToggleButton(
        value=False,
        description='Plot L-curve',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Plots L-curve to choose best value of regularization parameter of L2 reg. method',
        icon='plus')

    Heatplot = widgets.ToggleButton(
        value=False,
        description='Plot heatmap?',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Plots heatmap of data from choosed file',
        icon='plus')


    left_box = VBox([Methods, Nz, dt])
    centre_box = VBox([Reg_L1, Reg_L2, Reg_C, Reg_S, Bounds])
    right_box = VBox([Plot, LCurve, Residuals, Heatplot])
    ui = widgets.HBox([left_box, centre_box, right_box])
    Slider = widgets.HBox([Label('Transient №'),Index])
    out = widgets.interactive_output(demo, {'Index':Index,   'Nz':Nz, 
                                            'Reg_L1':Reg_L1, 'Reg_L2':Reg_L2, 'Reg_C':Reg_C, 'Reg_S':Reg_S, 
                                            'Bounds':Bounds, 'dt':dt,         'Methods':Methods,
                                            'Plot':Plot,     'LCurve':LCurve, 'Residuals':Residuals,
                                            'Heatplot': Heatplot})
    display(ui, Slider, out)


In [7]:
interface()

Output()